# PROJECT 5: Hospital Chatbot

### Name: Ojaswin Kohli

### Intern code: OGTIPOKDS440

### Batch: OGTIP - DS - AUS - 08

**Project Description:**

I have made a hospital chatbot for a renowned hospital providing first class assistance in peak hrs of the hospital.
I have made necessary changes to the dataset that included {Queries/response} collection of data to make an interactive chatbot data set. After refining the data set, I built a machine learning model that can generate response to an individuals query to help the hospital customer care department.



In [ ]:
# importing modules
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import save_model


# importing training data
# Load training data with context
training_data = pd.read_csv("/content/training_data_chat_bot.txt", usecols=["patterns", "tags", "context"])
training_data.sample(20)


,patterns,tags,context
23,How you can be helpful?,options,NaN
6,Bye,goodbye,NaN
53,NaN,search_hospital_by_type,NaN
52,NaN,search_hospital_by_params,search_hospital_by_type
0,Hi there,greeting,NaN
30,Open blood pressure module,blood_pressure,NaN
14,Awesome thanks,thanks,NaN
19,,noanswer,NaN
37,Load patient blood pressure result,blood_pressure_search,search_blood_pressure_by_patient_id
15,Thanks for helping me,thanks,NaN


In [ ]:
# preprocessing training data
training_data.fillna('', inplace=True)
training_data["patterns"] = training_data["patterns"].str.lower()
training_data["context"] = training_data["context"].str.lower()
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words="english")
training_data_tfidf = vectorizer.fit_transform(training_data["patterns"]).toarray()

# preprocessing target variable(tags)
le = LabelEncoder()
training_data_tags_le = pd.DataFrame({"tags": le.fit_transform(training_data["tags"])})
training_data_tags_dummy_encoded = pd.get_dummies(training_data_tags_le["tags"]).to_numpy()

# creating DNN
hospitalBot = Sequential()
hospitalBot.add(Dense(10, input_shape=(len(training_data_tfidf[0]),)))
hospitalBot.add(Dense(8))
hospitalBot.add(Dense(8))
hospitalBot.add(Dense(6))
hospitalBot.add(Dense(len(training_data_tags_dummy_encoded[0]), activation="softmax"))
hospitalBot.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

# fitting DNN
hospitalBot.fit(training_data_tfidf, training_data_tags_dummy_encoded, epochs=1000, batch_size=32)

# saving model file
save_model(hospitalBot, "chatbot_model_v1")

Epoch 1/1000
2/2 [==============================] - 1s 7ms/step - loss: 2.6312 - accuracy: 0.0556
Epoch 2/1000
2/2 [==============================] - 0s 9ms/step - loss: 2.6088 - accuracy: 0.0741
Epoch 3/1000
2/2 [==============================] - 0s 8ms/step - loss: 2.5923 - accuracy: 0.0926
Epoch 4/1000
2/2 [==============================] - 0s 9ms/step - loss: 2.5788 - accuracy: 0.1111
Epoch 5/1000
2/2 [==============================] - 0s 7ms/step - loss: 2.5661 - accuracy: 0.1111
Epoch 6/1000
2/2 [==============================] - 0s 8ms/step - loss: 2.5533 - accuracy: 0.0926
Epoch 7/1000
2/2 [==============================] - 0s 8ms/step - loss: 2.5419 - accuracy: 0.0926
Epoch 8/1000
2/2 [==============================] - 0s 7ms/step - loss: 2.5293 - accuracy: 0.0926
Epoch 9/1000
2/2 [==============================] - 0s 7ms/step - loss: 2.5184 - accuracy: 0.0926
Epoch 10/1000
2/2 [==============================] - 0s 8ms/step - loss: 2.5073 - accuracy: 0.1111
Epoch 11/1000
2/2 [

In [ ]:
# importing modules
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras.models import load_model
import json
import random
#Let’s load data, model, and responses

# importing training data
training_data = pd.read_csv("/content/training_data_chat_bot.txt", usecols=["patterns", "tags", "context"])

# Replace NaN values with empty strings in 'patterns' and 'context' columns
training_data.fillna('', inplace=True)

# loading model
chatbot_model = load_model("chatbot_model_v1")


In [ ]:
# loading responses
# responses = json.load(open("/content/response.json"))

In [ ]:
# Load responses
with open("/content/response.json") as file:
    responses = json.load(file)

In [ ]:
# fitting TfIdfVectorizer with training data to preprocess inputs
training_data["patterns"] = training_data["patterns"].str.lower()
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words="english")
vectorizer.fit(training_data["patterns"])

TfidfVectorizer(ngram_range=(1, 2), stop_words='english')

In [ ]:
# fitting LabelEncoder with target variable(tags) for inverse transformation of predictions
le = LabelEncoder()
le.fit(training_data["tags"])

LabelEncoder()

In [ ]:
# Create a dictionary to map intents to their follow-up contexts
intent_to_context = dict(zip(training_data["tags"], training_data["context"]))

# Context handling
current_context = ""

def update_context(tag):
    global current_context
    if tag in intent_to_context and intent_to_context[tag] != "":
        current_context = intent_to_context[tag]
    else:
        current_context = ""  # Reset context if no follow-up is required


In [ ]:
# transforming input and predicting intent
def predict_tag(inp_str):
    inp_data_tfidf = vectorizer.transform([inp_str.lower()]).toarray()
    predicted_proba = chatbot_model.predict(inp_data_tfidf)
    encoded_label = [np.argmax(predicted_proba)]
    predicted_tag = le.inverse_transform(encoded_label)[0]
    return predicted_tag

In [ ]:
# defining chat function
def start_chat():
    print("---------------  Welcome to our Hospital Assistance Chatbot  -------"
      "--------")
    print()
    print("Hi! I'm here to help you with your queries regarding our services "
      "and facilities.")
    print()
    print("You can ask me about appointments, health services, hospital "
      "information, and more.")
    print()
    print("Just type your question below to start. If at any point you wish to "
      "end the conversation, type 'EXIT'.")
    print()
    while True:
      inp = input("Ask anything... : ")
      if inp == "EXIT":
        break
      else:
        if current_context == "":
          tag = predict_tag(inp)
          update_context(tag)
          response = random.choice(responses[tag])
          print("Response... : ", response)
        else:
          print('',current_context)
          response = random.choice(responses[current_context])
          update_context(current_context)
          print("Response... : ", response)

# calling chat function to start chatting
start_chat()

---------------  Welcome to our Hospital Assistance Chatbot  ---------------

Hi! I'm here to help you with your queries regarding our services and facilities.

You can ask me about appointments, health services, hospital information, and more.

Just type your question below to start. If at any point you wish to end the conversation, type 'EXIT'.

Ask anything... : hey there how are you
1/1 [==============================] - 0s 75ms/step
Response... :  Hello, thanks for asking
Ask anything... : hi
1/1 [==============================] - 0s 32ms/step
Response... :  Hi there, how can I help?
Ask anything... : "I want to search hospital data
1/1 [==============================] - 0s 19ms/step
Response... :  Please provide hospital name or location
Ask anything... : Brisbane
 search_hospital_by_params
Response... :  Please provide hospital type
Ask anything... : Liver diseases
 search_hospital_by_type
Response... :  Loading hospital details
Ask anything... : List of pharmacies nearby
1/1 [=

Credits:

The code for this chatbot is built upon the edubot chatbot code from the Oeson (Simple_chatbot_DNN (5).ipynb)